## Модули

In [9]:
# Модель
import optuna as opt
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, make_scorer, confusion_matrix

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from helpers.data import DataPreprocessor
from helpers.validation import valid_predictions

# Данные
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Загрузка датасетов

In [ ]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
BALANCED_DATASET = os.path.join(ROOT, '../data/balanced_train.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')
SUBMISSION_PATH = os.path.join(ROOT, '../submissions/')

def make_predictions(model, X_test):
    # Загрузка сбалансированного датасета
    balanced_df = pd.read_csv(BALANCED_DATASET, index_col=0)

    # Удаление дубликатов
    balanced_df = balanced_df.drop_duplicates()

    # Обрезка негативных записей до числа позитивных
    first_negatives = balanced_df[balanced_df['y'] == 0][:balanced_df[balanced_df['y'] == 1]['y'].count()]
    balanced_df = pd.concat([balanced_df[balanced_df['y'] == 1], first_negatives])

    # Удаление выбросов
    balanced_df = balanced_df[balanced_df['Длительность'] < 400]
    balanced_df = balanced_df[(balanced_df['Сумма'] > 2) & (balanced_df['Сумма'] < 10)]
    balanced_df = balanced_df[balanced_df['До поставки'] < 300]
    balanced_df = balanced_df[balanced_df['Дней между 0_1'] < 400]
    balanced_df = balanced_df[balanced_df['Количество изменений после согласований'] < 2000]
    balanced_df = balanced_df[balanced_df['Количество'] < 300000]

    X, y = balanced_df.iloc[:, :-1], balanced_df.iloc[:, -1]   
    model.fit(X, y)
    preds = model.predict(X_test)

    save_submission(preds, 'submission')
    
def save_submission(preds, subname):
    subname = os.path.join(SUBMISSION_PATH, f'{subname}.csv')
    submit_df = pd.DataFrame({'id': test_df.index, 'value': preds})
    submit_df.to_csv(subname, index=False)

# Загрузка датасетов
train_df = pd.read_csv(TRAIN_DATASET)
test_df = pd.read_csv(TEST_DATASET)

# Удаление дубликатов из тренировочной выборки
train_df = train_df.drop_duplicates()

# Разделение выборки на тренировочную и тестовую
X, y = train_df.iloc[:, :-1], train_df.iloc[:, -1]   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Удаление выбросов из тренировочной выборки
X_train = pd.concat([X_train, y_train], axis=1) # Объединяем X_train и y_train для удаления выбросов только из тренировочной выборки
X_train = X_train[X_train['Длительность'] < 400]
X_train = X_train[(X_train['Сумма'] > 2) & (X_train['Сумма'] < 10)]
X_train = X_train[X_train['До поставки'] < 300]
X_train = X_train[X_train['Дней между 0_1'] < 400]
X_train = X_train[X_train['Количество изменений после согласований'] < 2000]
X_train = X_train[X_train['Количество'] < 300000]

# Отделяем метки классов от признаков
y_train = X_train['y']
X_train = X_train.drop('y', axis=1)

# Общий датасет для большего числа записей на кросс-валидации
X_general, y_general = train_df.iloc[:, :-1], train_df.iloc[:, -1]

## Препроцессор

In [ ]:
def objective_f1_macro(trial: opt.Trial) -> float:
    """ Функция оптимизации F1 (macro).
    
    Использует библиотеку optuna для подбора параметров из
    заданного диапазона. В качестве оптимизируемой метрики 
    выступает F1 (macro).
    
    Параметры:
        trial: экзмепляр optuna.Trial, представляющей собой
        историю оптимизации целевой функции.
        
    Пример:
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=100)
        
    Возвращает F1 (macro) метрику для подобранных параметров."""
    
    # Параметры
    learning_rate = trial.suggest_float('learning_rate', 0.01, 1)
    n_estimators = trial.suggest_int('n_estimators', 500, 3000)
    max_depth = trial.suggest_int('max_depth', 6, 32)
    max_bin = trial.suggest_int('max_bin', 32, 200),
    num_leaves = trial.suggest_int('num_leaves', 32, 300)
    reg_lambda = trial.suggest_float('reg_lambda', 0.01, 1)

    # Модель
    data_preprocessor = DataPreprocessor()
    model = LGBMClassifier(learning_rate=learning_rate, n_estimators=n_estimators,
                           max_depth=max_depth, max_bin=max_bin, num_leaves=num_leaves,
                           reg_lambda=reg_lambda)

    pipeline = Pipeline([
        ('data_preproc', data_preprocessor),
        ('model', model)
    ])
    
    cv_score = cross_val_score(pipeline, X, y, cv=StratifiedKFold(n_splits=5), scoring='f1_macro', n_jobs=-1)
    accuracy = cv_score.mean()

    return accuracy

In [ ]:
# Запуск подбора гиперпараметров
study = opt.create_study(direction='maximize')
study.optimize(objective_f1_macro, n_trials=100)

In [ ]:
def objective_confusion_matrix(trial: opt.Trial) -> int:
    """ Функция оптимизации FP и FN значений матрицы ошибок.
    
    Использует библиотеку optuna для подбора параметров из
    заданного диапазона. В качестве оптимизируемой метрики 
    выступает сумма FP и FN значений матрицы ошибок.
    
    Параметры:
        trial: Экзмепляр optuna.Trial, представляющей собой
            историю оптимизации целевой функции.
        
    Пример:
        study = optuna.create_study(direction='minimize')
        study.optimize(objective_confusion_matrix, n_trials=100)
        
    Возвращает сумму FP и FN значений матрицы ошибок для подобранных параметров."""
    
    # Параметры
    learning_rate = trial.suggest_float('learning_rate', 0.01, 1)
    n_estimators = trial.suggest_int('n_estimators', 500, 3000)
    max_depth = trial.suggest_int('max_depth', 6, 32)
    max_bin = trial.suggest_int('max_bin', 32, 200),
    num_leaves = trial.suggest_int('num_leaves', 32, 300)
    reg_lambda = trial.suggest_float('reg_lambda', 0.01, 1)


    # Модель
    data_preprocessor = DataPreprocessor()
    model = LGBMClassifier(learning_rate=learning_rate, n_estimators=n_estimators,
                           max_depth=max_depth, max_bin=max_bin, num_leaves=num_leaves,
                           reg_lambda=reg_lambda)
    
    pipeline = Pipeline([
        ('data_preproc', data_preprocessor),
        ('model', model)
    ])
    
    def confusion_matrix_score(y_true, y_pred):
        _, fp, fn, _ = confusion_matrix(y_true, y_pred).ravel()
        return fp + fn
    
    scorer = make_scorer(confusion_matrix_score)
    cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring=scorer)

    return cv_scores.mean()

In [ ]:
# Запуск подбора гиперпараметров
study = opt.create_study(direction='minimize')
study.optimize(objective_confusion_matrix, n_trials=100)

## Тесты

In [ ]:
# Параметры модели
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'force_col_wise': True,
    'learning_rate': 0.25595401399911066, 
    'n_estimators': 2122, 
    'max_depth': 14, 
    'max_bin': 163, 
    'num_leaves': 176, 
    'reg_lambda': 0.15394653650963538, 
    }

# Предобработчик и модель
data_preprocessor = DataPreprocessor()
model = LGBMClassifier(**parameters, n_jobs=-1)

# Пайплайн
pipeline = Pipeline([
    ('data_preproc', data_preprocessor),
    ('model', model)
])

### Кросс-валидация

In [ ]:
# Запуск кросс-валидации
cv_scores = cross_val_score(pipeline, X_general, y_general, cv=StratifiedKFold(n_splits=5), scoring='f1_macro', n_jobs=-1)
print(f'Mean CV: {cv_scores.mean()}')

### Валидация на тестовой выборке

In [ ]:
# Обучение модели и получение предсказаний для тестовой выборки
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)

In [ ]:
# Вывод результатов валидации
valid_predictions(y_test, preds)

### Важность признаков

In [ ]:
feature_importances = pd.DataFrame({'Feature': pipeline['model'].feature_name_, 'Importance': pipeline['model'].feature_importances_})
feature_importances.to_csv('features.csv', index=False)

## Финальный фит-предикт

In [ ]:
pipeline.fit(X_general, y_general)
save_submission(pipeline, 'submission')